In [1]:
# Disable Cache
import os, sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import quantfreedom as qf
import pandas as pd
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.base.base import rsi_below_entries
from quantfreedom.backtester.enums.enums import (
    LeverageMode, 
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
    )
# qf.clear_cache()

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[50],
    prices=prices.close
)

In [5]:
df, order_rec, cart = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[5],
    size_pct=1.,
    tsl_pcts=2,
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=1,
    tsl_when_pct_from_avg_entry=1,
)
pd.DataFrame(order_rec).head(10)


,avg_entry,bar,equity,fees_paid,ind_set,or_set,order_id,order_type,price,real_pnl,size_value,sl_prices,tp_prices,tsl_prices
0,10850.000000,31,1000.0,NaN,0,0,0,0.0,10850.000000,NaN,471.965263,NaN,12013.166900,10633.000000
1,10787.137891,32,1000.0,NaN,0,0,1,0.0,10725.000000,NaN,471.965263,NaN,11943.565701,10571.395133
2,10787.137891,32,980.0,1.121389,0,0,2,6.0,10571.395133,-20.0,943.930527,NaN,11943.565701,10571.395133
3,10577.500000,33,980.0,NaN,0,0,3,0.0,10577.500000,NaN,462.525958,NaN,11711.453722,10365.950000
4,10550.179617,34,980.0,NaN,0,0,4,0.0,10523.000000,NaN,462.525958,NaN,11681.204476,10339.176024
5,10538.761751,35,980.0,NaN,0,0,5,0.0,10516.000000,NaN,462.525958,NaN,11668.562565,10327.986516
6,10538.761751,571,1127.0,1.754346,0,0,6,5.0,11668.562565,147.0,1387.577874,NaN,11668.562565,10327.986516
7,11535.000000,577,1127.0,NaN,0,0,7,0.0,11535.000000,NaN,531.904852,NaN,12771.601861,11304.300000
8,11517.974909,578,1127.0,NaN,0,0,8,0.0,11501.000000,NaN,531.904852,NaN,12752.751607,11287.615411
9,11516.316128,580,1127.0,NaN,0,0,9,0.0,11513.000000,NaN,531.904852,NaN,12750.914998,11285.989806


In [3]:
pd.DataFrame(df)

,or_set,ind_set,total_trades,total_BE,gains_pct,win_rate,to_the_upside,total_fees,total_pnl,ending_eq
0,0,0,366.0,0,-18.983015,11.75,0.714017,NaN,-189.8298,810.169853


In [4]:
pd.DataFrame(cart).T

,0
leverage,NaN
max_equity_risk_pct,3.0
max_equity_risk_value,NaN
risk_rewards,5.0
size_pct,1.0
size_value,NaN
sl_pcts,NaN
sl_prices,NaN
sl_to_be,False
sl_to_be_based_on,NaN
